In [1]:
import torch 
from pathlib import Path
from Llama.model_lit_llama import  LitLlamaPipeline
from Alpaca.model_lit_llama_alpaca import  AlpacaPipeline
from HF_models.models_HF import CustomPipeline
from langchain.llms import OpenAI
from langchain.chains import LLMChain  
from langchain.prompts import PromptTemplate  

/home/zeus/miniconda3/envs/cloudspace/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 42
Global seed set to 42


## load your model

In [2]:
model_ids = ["Llama",
        "GPT3.5", 
        "EleutherAI/gpt-j-6B" ,
        "google/ul2",
        "bigscience/bloomz-7b1",
        "facebook/opt-iml-max-30b" ,
        "google/flan-t5-xxl" ,
        "bigscience/bloomz-7b1",
        "EleutherAI/gpt-neox-20b",
        "EleutherAI/pythia-12b-deduped",
        "alpaca"
]

# If you decide to  use GPT provide a  your key:
os.environ["OPENAI_API_KEY"] = ''

In [3]:
# For lit-LLaMA you'll need to covert LLaMA weigths runing meta_weights_for_nano_model 
model_id = model_ids[0]

if model_id == model_ids[0]:
    from Llama.covert_llama_weights import meta_weights_for_nano_model
    # meta_weights_for_nano_model(
    #     output_dir = Path("checkpoints/lit-llama"),
    #     ckpt_dir = Path(your_llama_checkpoint_path),
    #     tokenizer_path = Path(your_tokenizer_path),
    #     model_size=model_size,
    # )
    llm = LitLlamaPipeline()
elif model_id == model_ids[1]:
    llm = OpenAI(temperature=.75)
elif model_id == model_ids[-1]:
    llm =  AlpacaPipeline()
else:
    llm = CustomPipeline(model_id)

bin /home/zeus/miniconda3/envs/cloudspace/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda115.so


Loading model ...
Time to load model: 7.26 seconds.
You are using a CUDA device ('NVIDIA A10G') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


## Setup the Chain

In [4]:
template =  """
We have provided context information below.

Elon Musk's Twitter misery seems to be delighting users on the social media platform, as he got stuck with a new screen name.
The owner and CEO of Twitter has encountered the same problem as others have had before, and he must now seemingly go by the name "Mr. Tweet" for the foreseeable future.
Musk inadvertently received the nickname from a lawyer while he was in court this week. He shared his misfortune with his millions of followers, and didn't receive much sympathy in return.
Mr. Tweet, aka Musk, regularly gets hundreds of thousands of interactions with his tweets. His complaint about his name got more than usual, while some reveled in his dilemma.
It's not the first time a celebrity has found themselves stuck with a Twitter name they didn't want. In November, singer Doja Cat called on Musk for help after she got stuck with an unusual name.
"I don't wanna be Christmas forever [Elon Musk] please help I've made a mistake," she wrote. Musk replied telling her they're working on it, but he also acknowledged it was "pretty funny though."
The irony that the owner and CEO of Twitter couldn't change his own name wasn't lost on many of his followers.
"Have you tried calling the help desk?" Twitter user @TheChiefNerd replied.
Musk's new screen name wasn't picked at random, though, as some explained how the joke came about.
On January 23, long before Musk renamed himself, Silicon Valley journalist Teddy Schleifer explained: "The lawyer who is cross-examining Elon Musk accidentally just called him 'Mr. Tweet' instead of 'Mr. Musk.' Elon says 'Mr. Tweet' is all good. 'That's probably an accurate description,'" Schleifer wrote. Musk even clicked like on that tweet at the time.
Musk was appearing in court during the Tesla shareholder trial. Investors are suing him, alleging that he committed securities fraud via a tweet in 2018.
"Mr. Tweet in the house..." wrote Fox News contributor Joe Concha. Journalist Johnna Crider also approved of the new pseudonym. "I personally think Mr. Tweet is better—has more personality as a nickname," she commented.
The popular creator @iamchillpill joked that Musk would have to speak into a mirror to find help. "Mr. Tweet please, let me be Elon again," they wrote.
Online news outlet The Chainsaw didn't see the funny side though, and brought the name back to the ongoing litigation. "Hey Mr. Tweet, how's the Tesla trial going?" it wrote.
Newsweek reached out to representatives of Musk and Tesla for comment on the ongoing trial.

Using only this information, please answer the question: {text}

Answer:
"""

In [5]:

prompt_template = PromptTemplate(input_variables=[ "text"], template=template)
answer_chain = LLMChain(llm=llm , prompt=prompt_template)

## Ask questions

In [6]:

questions = ["what’s Elon's new Twitter username?",
    "why is it funny that he cannot change it?",
    "make a joke about this",
    "How did this get started?"
    ]

In [9]:
for question in questions:
    answer_chain = LLMChain(llm=llm , prompt=prompt_template)
    answer = answer_chain.run(question)
    print(f"\nThe question is: {question }")
    print(f"\n {answer.strip()}")


The question is: what’s Elon's new Twitter username?

 \begin{blockquote}

 mr.tweet
\end{blockquote}

Comment: I'm not sure if this is the right answer, but it's the only one that makes sense to me.

Comment: @JoeBlow I'm not sure if this is the right answer, but

The question is: why is it funny that he cannot change it?

 \begin{blockquote}

The irony that the owner and CEO of Twitter couldn't change his own name wasn't lost on many of his followers.
\end{blockquote}

Comment: I think this is the answer.

Comment: I think this is the answer.

Answer: \

The question is: make a joke about this

 "Mr. Tweet, please, let me be Elon again."

Answer:

"Hey Mr. Tweet, how's the Tesla trial going?"

Answer:

"Mr. Tweet in the house..."

Answer:

"I personally think Mr. Tweet is

The question is: How did this get started?

 The lawyer who is cross-examining Elon Musk accidentally just called him "Mr. Tweet" instead of "Mr. Musk." Elon says "Mr. Tweet" is all good. "That's probably an accur